<a href="https://colab.research.google.com/github/gayatribandaru/Apache-Beam-data-engineering-assignment/blob/main/Datamining_Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load the data from the CSV file
data = pd.read_csv('/content/hungary_chickenpox.csv')

# Display the first few rows of the dataset
data.head()


,Date,BUDAPEST,BARANYA,BACS,BEKES,BORSOD,CSONGRAD,FEJER,GYOR,HAJDU,...,JASZ,KOMAROM,NOGRAD,PEST,SOMOGY,SZABOLCS,TOLNA,VAS,VESZPREM,ZALA
0,03/01/2005,168,79,30,173,169,42,136,120,162,...,130,57,2,178,66,64,11,29,87,68
1,10/01/2005,157,60,30,92,200,53,51,70,84,...,80,50,29,141,48,29,58,53,68,26
2,17/01/2005,96,44,31,86,93,30,93,84,191,...,64,46,4,157,33,33,24,18,62,44
3,24/01/2005,163,49,43,126,46,39,52,114,107,...,63,54,14,107,66,50,25,21,43,31
4,31/01/2005,122,78,53,87,103,34,95,131,172,...,61,49,11,124,63,56,7,47,85,60


In [ ]:
pip install apache_beam


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 18.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.3/671.3 kB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 55.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 26.5 M

In [ ]:
!pip install apache_beam


In [ ]:
from apache_beam.transforms.window import FixedWindows
from apache_beam.transforms.trigger import Repeatedly, AfterCount

In [ ]:
# Define the windowing and triggering configurations
monthly_window = FixedWindows(30 * 24 * 60 * 60)  # Monthly window in seconds
trigger = Repeatedly(AfterCount(4))


In [ ]:
class CalculateMonthlyTotal(beam.PTransform):
    def expand(self, pcoll):
        return (pcoll
                | 'ParseRow' >> beam.ParDo(ParseRow())
                | 'WindowInto' >> beam.WindowInto(monthly_window, trigger=trigger)
                | 'Aggregate' >> beam.CombinePerKey(sum))


In [ ]:
import apache_beam as beam
from apache_beam.io import ReadFromText, WriteToText
from apache_beam.transforms.window import FixedWindows
from apache_beam.transforms.trigger import AfterWatermark, AfterCount
from apache_beam.options.pipeline_options import PipelineOptions

# Define the ParseRow DoFn
class ParseRow(beam.DoFn):
    def process(self, row):
        fields = row.split(',')
        date = fields[0]

        # Skip header row
        if date == "Date":
            return

        counts = [int(x) for x in fields[1:]]
        for idx, count in enumerate(counts):
            yield (idx, count)


# Define the windowing and triggering configurations
monthly_window = beam.WindowInto(FixedWindows(30 * 24 * 60 * 60))  # Monthly window in seconds
trigger = AfterWatermark(early=AfterCount(4))

# Define the composite transform
class CalculateMonthlyTotal(beam.PTransform):
    def expand(self, pcoll):
        return (pcoll
                | 'ParseRow' >> beam.ParDo(ParseRow())
                | 'WindowInto' >> monthly_window
                | 'Aggregate' >> beam.CombinePerKey(sum))

# Create and run the Beam pipeline
options = PipelineOptions()
with beam.Pipeline(options=options) as p:
    (p | 'ReadData' >> ReadFromText('/content/hungary_chickenpox.csv')
       | 'ProcessData' >> CalculateMonthlyTotal()
       | 'WriteResults' >> WriteToText('path_to_output_file.txt'))


In [ ]:
import glob

# Read and display the output files
output_files = glob.glob('path_to_output_file.txt*')
for file in output_files:
    with open(file, 'r') as f:
        print(f.read())


(0, 52850)
(1, 17855)
(2, 19401)
(3, 15092)
(4, 29797)
(5, 16437)
(6, 17368)
(7, 21630)
(8, 24585)
(9, 15499)
(10, 21334)
(11, 13386)
(12, 11406)
(13, 44945)
(14, 14412)
(15, 15584)
(16, 10624)
(17, 11728)
(18, 21212)
(19, 10374)

